In [1]:
from vivarium_public_health.interactive import setup_simulation_from_model_specification

sim = setup_simulation_from_model_specification('/home/kate/code/vivarium/neonatal/neonatal/model_specs/test_lbwsg.yaml')

/home/kate/miniconda3/envs/neonatal/lib/python3.6/site-packages/vivarium/framework/configuration.py:35: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml.load(file_path)
/home/kate/miniconda3/envs/neonatal/lib/python3.6/site-packages/vivarium/config_tree.py:394: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data_dict = yaml.load(data_string)
/home/kate/miniconda3/envs/neonatal/lib/python3.6/site-packages/vivarium_public_health/population/data_transformations.py:156: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  pop_data = pd.concat([lower_bi

In [2]:
lbwsg = sim.get_value('low_birth_weight_and_short_gestation.exposure')

pop_start = sim.get_population()
import time
s = time.time()
at_start = lbwsg(pop_start.index)
e = time.time()
print(f'Calling lbwsg exp pipeline took {e - s} seconds.')

Calling lbwsg exp pipeline took 2.4388952255249023 seconds.


In [ ]:
at_start.value_counts()

In [ ]:
pop_start.age.hist()

In [3]:
from vivarium_public_health.dataset_manager import Artifact
art = Artifact(sim.configuration.input_data.artifact_path, 
               filter_terms=[f'draw == {sim.configuration.input_data.input_draw_number}'])
exp = art.load('risk_factor.low_birth_weight_and_short_gestation.exposure')

In [4]:
age_group_start = 0
age_group_end = 7/365
sex = 'Female'

pop_data = pop_start[(pop_start.age < age_group_end) & (pop_start.age >= age_group_start) 
                     & (pop_start.sex == sex)]
art_data = exp[(exp.age_group_start == age_group_start) & (exp.year_start == sim.clock.time.year) \
               & (exp.sex == sex)]

comp = art_data[['parameter', 'value']].set_index('parameter').rename(columns={'value': 'art'})
comp = comp.join((lbwsg(pop_data.index).value_counts()/len(pop_data.index)).rename('sim'))
comp['diff'] = (comp.art - comp.sim) / comp.art
comp

,art,sim,diff
parameter,,,
cat10,0.001619,0.001361,0.159166
cat106,0.001242,0.002722,-1.191110
cat11,0.001187,0.001361,-0.146714
cat116,0.002362,0.002722,-0.152712
cat117,0.009180,0.011797,-0.285005
cat123,0.002918,0.004991,-0.710641
cat14,0.001502,0.000907,0.395739
cat15,0.001652,0.001361,0.175945
cat17,0.003204,0.002269,0.291894


In [5]:
pop_data.shape

(2204, 9)

In [ ]:
#sim.run()